In [9]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [17]:
OPENAI_API_KEY='sk'
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [18]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

In [19]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/Aseer"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/Riyadh"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [11]:
from llama_index.readers.file.docs.base import DocxReader, HWPReader

In [20]:
if not index_loaded:
    # load data
    Aseer_docs = SimpleDirectoryReader(
        input_files=["Evolution-of-Population-in-Aseer-by-Gender.pdf"]
    ).load_data()
    Riyadh_docs = SimpleDirectoryReader(
        input_files=["Evolution-of-Population-in-Al-Riyadh-by-Gender.pdf"]
    ).load_data()

    # build index
    Aseer_index = VectorStoreIndex.from_documents(Aseer_docs)
    Riyadh_index = VectorStoreIndex.from_documents(Riyadh_docs)

    # persist index
    Aseer_index.storage_context.persist(persist_dir="./storage/Aseer")
    Riyadh_index.storage_context.persist(persist_dir="../storage/Riyadh")

In [21]:
Aseer_index

In [22]:
Aseer_engine = Aseer_index.as_query_engine(similarity_top_k=3)
Riyadh_engine = Riyadh_index.as_query_engine(similarity_top_k=3)

In [23]:
Aseer_engine

In [26]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=Aseer_engine,
        metadata=ToolMetadata(
            name="aseer_population",
            description=(
                "Provides information about Aseer population from 2010 to 2022 "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=Riyadh_engine,
        metadata=ToolMetadata(
            name="riyadh_population",
            description=(
                "Provides information about Riyadh population from 2010 to 2022 "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [24]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [29]:

llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

response = agent.chat("What was Aseer population growth percentage of females from 2012 to 2014")
print(str(response))

> Running step 46bf5a2e-49b9-42f9-b5c4-a67e363a6344. Step input: What was Aseer population growth percentage of females from 2012 to 2014
Thought: The user is asking about the population growth percentage of females in Aseer from 2012 to 2014. I can use the "aseer_population" tool to get the population data for these years and calculate the growth percentage.
Action: aseer_population
Action Input: {'input': 'population growth percentage of females from 2012 to 2014'}
Observation: The population growth percentage of females from 2012 to 2014 is 5.8%.
> Running step 20e1d8eb-e7db-4543-aa24-37fb3a916882. Step input: None
Thought: I have obtained the information about the population growth percentage of females in Aseer from 2012 to 2014. I can now provide the answer to the user's question.
Answer: The population growth percentage of females in Aseer from 2012 to 2014 is 5.8%.
The population growth percentage of females in Aseer from 2012 to 2014 is 5.8%.


In [32]:
response = agent.chat(
    "Compare and contrast the total population growth of Aseer and Riyadh from 2016 till 2022, give the percentage of growth, then"
    " give an analysis"
)
print(str(response))

> Running step 3ea09f42-c8a3-4920-af8a-d1e999887860. Step input: Compare and contrast the total population growth of Aseer and Riyadh from 2016 till 2022, give the percentage of growth, then give an analysis
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: aseer_population
Action Input: {'input': 'total population from 2016 to 2022'}
Observation: The total population from 2016 to 2022 in Aseer was 1863056, 1879189, 1863690, 1876328, 1958677, 1943532, and 2024285 respectively.
> Running step ac6f090b-5997-4f9c-87ff-a7103360720a. Step input: None
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: riyadh_population
Action Input: {'input': 'total population from 2016 to 2022'}
Observation: The total population from 2016 to 2022 in Al-Riyadh is 8,271,126 in 2016, 8,275,538 in 2017, 8,030,674 in 2018, 7,979,262 in 2019, 8,422,408 in 2020, 8,175,378 in 2021, and 